In [1]:
from wikibaseintegrator.models import Qualifiers, References, Reference
from wikibaseintegrator import WikibaseIntegrator
from wikibaseintegrator import wbi_login
from wikibaseintegrator import datatypes
from wikibaseintegrator.wbi_config import config as wbi_config
from os.path import join, isfile, exists, isdir, abspath, pardir
from os import listdir, getcwd
import pandas as pd
from wikibaseintegrator.datatypes import (URL, CommonsMedia, ExternalID, Form, GeoShape, GlobeCoordinate, Item, Lexeme, Math, MonolingualText, MusicalNotation, Property, Quantity, Sense, String, TabularData, Time)
from wikibaseintegrator.wbi_enums import ActionIfExists, WikibaseDatePrecision, WikibaseRank, WikibaseSnakType
from wikibaseintegrator.wbi_exceptions import MissingEntityException, ModificationFailed, MWApiError

In [2]:
# Default variables
WDUSER, WDPASS = "Admin", "90J8XXXaO4Sr9^^Z"
# WDUSER, WDPASS = "Mohammadzainabbas", "fHh!%shFa6^h"

In [3]:
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, "data")
column_mapping = join(data_dir, "column_mapping.csv")
project_mapping = join(data_dir, "project_mapping.csv")
team_mapping = join(data_dir, "team_mapping.csv")

In [4]:
def setup_config():
    """
    Set up WBI config to use local docker installation
    """
    wbi_config['MEDIAWIKI_API_URL'] = 'http://139.144.66.193:8181/api.php'
    wbi_config['SPARQL_ENDPOINT_URL'] = '"http://139.144.66.193:8282/proxy/wdqs/bigdata/namespace/wdq/sparql"'
    wbi_config['WIKIBASE_URL'] = 'http://wikibase.svc'

In [5]:
setup_config()

In [6]:
login = wbi_login.Login(user=WDUSER, password=WDPASS)

In [7]:
wbi = WikibaseIntegrator(login=login)

In [14]:
df = pd.read_csv(column_mapping)
df.head()

,column,prop,data_type,type,mapping_code
0,Id,P1,string,internal,NaN
1,Name,P2,external-id,external,P735
2,Family Name,P3,external-id,external,P734
3,Birthdate,P4,external-id,external,P569
4,Nationality,P5,external-id,external,P27


In [18]:
column_mapping = df.set_index("column").T.to_dict()

In [21]:
column_mapping['Project End date']

{'prop': 'P18',
 'data_type': 'external-id',
 'type': 'external',
 'mapping_code': 'P582'}